In [1]:
from vatic.engines import Simulator
from vatic.data.loaders import load_input, RtsLoader, T7kLoader
from vatic.engines import Simulator
from pgrisk.cost_attributor import CostAttributor

import pandas as pd
import numpy as np
from functools import reduce

from pathlib import Path
import dill as pickle
import os
from datetime import datetime
import matplotlib.pyplot as plt

RUC_MIPGAPS = {'RTS-GMLC': 0.01}
SCED_HORIZONS = {'RTS-GMLC': 4}

## Run unit commitment simulation

In [2]:
grid = 'RTS-GMLC'  
num_days = 1  
start_date = '2020-01-15'
init_state_file = None
template, gen_data, load_data = load_input(grid, start_date, 
    num_days=num_days, init_state_file=init_state_file)

data_dir = Path('data', start_date)
os.makedirs(data_dir, exist_ok=True)
ruc_out = Path(data_dir, 'rts-gmlc-ruc.p')

siml = Simulator(template, gen_data, load_data, None,
                pd.to_datetime(start_date).date(), 1, solver='gurobi',
                solver_options={}, run_lmps=False, mipgap=RUC_MIPGAPS[grid],
                load_shed_penalty = 1e4, reserve_shortfall_penalty = 1e3,
                reserve_factor=0.05, output_detail=3,
                prescient_sced_forecasts=True, ruc_prescience_hour=0,
                ruc_execution_hour=16, ruc_every_hours=24,
                ruc_horizon=48, sced_horizon=SCED_HORIZONS[grid],
                lmp_shortfall_costs=False,
                enforce_sced_shutdown_ramprate=False,
                no_startup_shutdown_curves=False,
                init_ruc_file=ruc_out, verbosity=0,
                output_max_decimals=4, create_plots=False,
                renew_costs=None, save_to_csv=False, 
                last_conditions_file=None,)
_ = siml.simulate()


(ordered) position is deprecated.  Please use at()  (deprecated in 6.1, will
be removed in (or after) 7.0) (called from
/home/xy3134/.conda/envs/pgrisk_dev/lib/python3.9/site-
packages/vatic/models/params.py:1022)
(ordered) position is deprecated.  Please use at()  (deprecated in 6.1, will
be removed in (or after) 7.0) (called from
/home/xy3134/.local/lib/python3.9/site-
packages/egret/model_library/unit_commitment/startup_costs.py:126)
(ordered) position is deprecated.  Please use at()  (deprecated in 6.1, will
be removed in (or after) 7.0) (called from
/home/xy3134/.local/lib/python3.9/site-
packages/egret/model_library/unit_commitment/startup_costs.py:127)
(ordered) position is deprecated.  Please use at()  (deprecated in 6.1, will
be removed in (or after) 7.0) (called from
/home/xy3134/.local/lib/python3.9/site-
packages/egret/model_library/unit_commitment/startup_costs.py:130)
Calculating PTDF Matrix Factorization
ignored.  (deprecated in 6.6.0) (called from
/home/xy3134/.local/li

## Compute cost attribution between baseline and target
#### baseline uses day-ahead forecast as inputs
#### target uses the actual data as inputs

In [3]:
baseline_gen_data = gen_data.copy(deep=True)
baseline_load_data = load_data.copy(deep=True)

for gen in baseline_gen_data.columns.get_level_values(1).unique():
    baseline_gen_data['actl', gen] = baseline_gen_data['fcst', gen]
for bus in baseline_load_data.columns.get_level_values(1).unique():
    baseline_load_data['actl', bus] = baseline_load_data['fcst', bus]

target_gen_data = gen_data.copy(deep=True)
target_load_data = load_data.copy(deep=True)

The cost attribution (code block below) can run at most 64 SCEDs and take long to finish.

In [4]:
workdir = Path(data_dir, 'working')
renewable_gens = list(template['ForecastRenewables'])
load_buses = template['Buses']

attr = CostAttributor(template_data = template,
                start_date = pd.to_datetime(start_date), mipgap = 0.01,
                reserve_factor = 0.05, lmp_shortfall_costs = False,
                init_ruc_file = ruc_out, verbosity = 2,
                renewable_gens = renewable_gens, load_buses = load_buses,
                baseline_gen_data = baseline_gen_data, 
                baseline_load_data = baseline_load_data, 
                target_gen_data = target_gen_data, 
                target_load_data = target_load_data,
                load_shed_penalty = 1e3, reserve_shortfall_penalty = 0.,
                workdir = Path(workdir), scale = 6, tol = 0.1,)
attr.run_shadow_price_analysis()
attr.compute_attribution()

cost attribution of renewable generator "309_WIND_1" and bus "Abel".

In [13]:
attr.cost_attribution['309_WIND_1'].head()

,costs
2020-01-15 00:00:00,1827.758070
2020-01-15 01:00:00,432.568825
2020-01-15 02:00:00,-434.038682
2020-01-15 03:00:00,-384.668679
2020-01-15 04:00:00,-2110.157306


In [14]:
attr.cost_attribution['Abel'].head()

,costs
2020-01-15 00:00:00,-69.058789
2020-01-15 01:00:00,-66.582990
2020-01-15 02:00:00,-35.190166
2020-01-15 03:00:00,-66.843529
2020-01-15 04:00:00,-84.716600


Attribution summary

In [16]:
attr.attribution_summary

,total_costs,attribution,renewable_costs,load_costs,unpriced_thermal_costs,initial_thernal_costs
Time,,,,,,
2020-01-15 00:00:00,15278.09,15112.06,17126.38,-2014.31,0.0,0.0
2020-01-15 01:00:00,9100.76,8565.91,10499.61,-1933.70,0.0,0.0
2020-01-15 02:00:00,6404.16,5831.48,6851.74,-1020.25,0.0,0.0
2020-01-15 03:00:00,10160.66,10358.22,12285.08,-1926.86,0.0,0.0
2020-01-15 04:00:00,2310.13,2826.30,5252.47,-2426.17,0.0,0.0
2020-01-15 05:00:00,-15512.45,-15016.57,-12030.42,-2986.15,0.0,0.0
2020-01-15 06:00:00,-8167.16,-8804.86,-5578.89,-3225.98,0.0,0.0
2020-01-15 07:00:00,2686.31,2131.04,4812.03,-2680.99,0.0,0.0
2020-01-15 08:00:00,0.00,0.00,0.00,0.00,0.0,0.0
